In [39]:
import pandas as pd
import openpyxl as exc
from openpyxl import workbook as wb
from openpyxl import load_workbook
import xlrd
import pyexcel as p
import os
from io import StringIO

In [29]:
def job_date(df):
    job = str(df.columns[0]).split(' ', 1)[1]
    edit_date = pd.to_datetime(str(df.iloc[0,0]).split(' ')[0])
    return job, edit_date



In [3]:
completed_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections"
on_going_path = ''

In [4]:
test_path = r"C:\Users\willd\OneDrive\Desktop\Belair\projection_analysis\completed proj projections\Projection 10378 Brunner Farm-CGRS 20231128.xlsx"

## handle the 2 sheet 30-50 issues

In [81]:
xl = pd.read_excel(test_path)
df = pd.DataFrame(xl)
job, date = job_date(df)
df.columns = df.iloc[2]
df = df.drop(df.index[:3])
df['updated'] = date
df['job'] = job
# remove non numeric values in this column
df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)

In [90]:
df_list = []
for item in os.listdir(completed_path):
    full_path = os.path.join(completed_path, item)
    num_sheet = len(pd.ExcelFile(full_path).sheet_names)
    for sheet in range(0,num_sheet):
        if 'Phase Code' in df.iloc[:,0]:
            xl = pd.read_excel(full_path ,sheet_name=sheet)
            df = pd.DataFrame(xl)
            job, date = job_date(df)
            df.columns = df.iloc[2]
            df = df.drop(df.index[:3])
            df['updated'] = date
            df['job'] = job
            # remove non numeric values in this column
            df = df[df['Phase Code'].apply(lambda x: str(x).isnumeric())]
            df['HJ Qty Comp'] = df['HJ Qty Comp'].fillna(0)
            df_list.append(df)
full_df = pd.concat(df_list)    



ValueError: No objects to concatenate

In [ ]:
full_path

In [87]:
if 'Phase Code' in df.iloc[:,0]:
    print(1)

In [85]:
df.iloc[:, 0]

3     1000
4     1001
5     1005
6     1010
7     1020
8     1030
9     1035
10    1040
11    1050
12    1060
13    1070
14    1080
15    1095
16    7020
17    7091
18    7092
19    7099
20    7105
21    7170
Name: Phase Code, dtype: object

In [28]:
print(job)
print(date)

10378 Brunner Farm-CGRS
2023-11-28 00:00:00


In [76]:
df.head(5)

,Projection 10292-50 Sorrel Ranch - Lokal,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,Date & Attendees,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Current Projection 2/7/20,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Phase Code,Description,Notes,Qty,HJ Qty Comp,Unit,% Comp,NaN,Budget Last Projection,New Budgets,...,Total New Costs,NaN,VP + HJ Cost to Date,NaN,New Projected Budget,Budget Remaining,Over/ (Under) Budget,% Comp,NaN,Over/ (Under) Variance
3,1001,Contingency,NaN,1,NaN,LS,0,NaN,18877.26,NaN,...,0,NaN,0,NaN,18877.26,18877.26,0,0,NaN,0
4,1005,Travel/Per Diem/Field Office,NaN,1,NaN,LS,0,NaN,2400,NaN,...,0,NaN,0,NaN,2400,2400,0,0,NaN,0


In [63]:
len(df.columns)

32